### Objective:
    Given is the nutritional value data of the Mcdonalds Menu in the United States.
    Using the said data the following objectives are to be archived:
        1. Identify a business question to be answered
        2. Identify a target variable to be predicted.
        2. Identify the features to be used to best predict the target variable.
        3. Implement a model to predict the target variable
#### High Level Tasks:
    1. Import and read data--Done
    2. Clean data and treat for any missing/non consistant values-WIP
        1.No Missing Values Found
        2 Got the Magnitude and Unit for each serving.
        3.Added a Cnt Column equal to 1 for each record
        4.Made all the Countinous data the proper data types
        5.aligned all the columns to the left and continous to the right
        6.Saved the dataset as csv as a check point.
        7.Sent the project to the Team for review.
    3. Explore data using discrete statistic to understand the data bettter--TBD
    

In [1]:
# import all the supporting libaries
import pandas as pb #using pandas as it gives the flexibility of working and altering the data in a data frame

### Import Data

In [196]:
del df
#Import and read data
df = pb.read_csv('McDonald_Menu.csv')

#### Clean up Column Names

In [197]:
#Clean data and treat for any missing/non consistant values
#Shorten Column Name to be more easy to read data

#reference : https://stackoverflow.com/questions/39741429/pandas-replace-a-character-in-all-column-names
#remove spaces from the column names
df.columns=df.columns.str.replace(" ", "_")


#replace _(%_Daily_Value) with  PrDV
df.columns=df.columns.str.replace("%_Daily_Value", "PrDV")
df.columns=df.columns.str.replace("[()]", "")

#Remove all the vowels in the column names
vowels=['a','e','i','o','u']
colmap = {}

for col in df.columns:
    col1=col
    for v in vowels:
        col1=col1.replace(v,'')
    colmap[col] = col1
df.columns = list(colmap.values())

#### Get the magniture and Unit for each serving

In [220]:
#Add Column serving magnitude and unit
    #Need to use the apply() function 
#Reference https://www.youtube.com/watch?v=eMOA1pPVUc4 [39:01]

#define funcition apply to get serviing size.
def serving_size_parse(ls_seving):
    mag_unit=[]
    serv_split = ls_seving.split()
    if 'g' == serv_split[-1].replace(')',''):
        mag = serv_split[-2].replace('(','')
        unit = serv_split[-1].replace(')','')
    elif 'cup' == serv_split[-1] :
        mag = serv_split[0]
        unit = serv_split[1] + serv_split[2]
    elif 'oz' == serv_split[-1]:
        mag = serv_split[0]
        unit = serv_split[1] + serv_split[2]
    elif 'carton' == serv_split[1]:
        mag = serv_split[-2].replace('(','')
        unit = serv_split[-1].replace(')','')
    elif  'fl' == serv_split[1]:
        mag = serv_split[-2].replace('(','')
        unit = serv_split[-1].replace(')','')
    else:
        mag = ''
        unit = ''
    mag_unit.append(ls_seving)
    mag_unit.append(mag)
    mag_unit.append(unit)
    return mag_unit

#Add Column serving magnitude and unit
#Populate the value using apply
df['Srvng_mag'] = 'Unknown'
df['Srvng_unit'] = 'Unknown'
df['Srvng_mag'] = df['Srvng_Sz'].apply(lambda x: serving_size_parse(x)[1])
df['Srvng_unit'] = df['Srvng_Sz'].apply(lambda x: serving_size_parse(x)[2])

#### Check for Missing Value

In [219]:
#Check for NaN and Null-No Missing Values Found
#Reference https://www.youtube.com/watch?v=eMOA1pPVUc4 [20:40]
nan_df = df[df.isna().any(axis=1)]
nan_df.head()
nan_df1 = df[df.isnull().any(axis=1)]
nan_df1.head()

,Ctgry,Itm,Srvng_Sz,Clrs,Clrs_frm_Ft,Ttl_Ft,Ttl_Ft_PrDV,Strtd_Ft,Strtd_Ft_PrDV,Trns_Ft,...,Dtry_Fbr,Dtry_Fbr_PrDV,Sgrs,Prtn,Vtmn_A_PrDV,Vtmn_C_PrDV,Clcm_PrDV,Irn_PrDV,Srvng_mag,Srvng_unit


#### Set the data proper datatype for the columns

In [239]:
#Loop Through all cols and set datatype as needed
#Reference : https://stackoverflow.com/questions/15891038/change-data-type-of-columns-in-pandas
    #https://www.w3resource.com/pandas/dataframe/dataframe-dtypes.php
numeric_col_ls = ['Clrs','Clrs_frm_Ft','Ttl_Ft','Ttl_Ft_PrDV','Strtd_Ft','Strtd_Ft_PrDV','Trns_Ft','Chlstrl','Chlstrl_PrDV','Sdm','Sdm_PrDV','Crbhydrts','Crbhydrts_PrDV','Dtry_Fbr','Dtry_Fbr_PrDV','Sgrs','Prtn','Vtmn_A_PrDV','Vtmn_C_PrDV','Clcm_PrDV','Irn_PrDV','Srvng_mag']
print(numeric_col_ls)

for col in df.columns:
    if col in numeric_col_ls:
        pb.to_numeric(df[col])
pb.to_numeric(df['Srvng_mag'])
df.dtypes


['Clrs', 'Clrs_frm_Ft', 'Ttl_Ft', 'Ttl_Ft_PrDV', 'Strtd_Ft', 'Strtd_Ft_PrDV', 'Trns_Ft', 'Chlstrl', 'Chlstrl_PrDV', 'Sdm', 'Sdm_PrDV', 'Crbhydrts', 'Crbhydrts_PrDV', 'Dtry_Fbr', 'Dtry_Fbr_PrDV', 'Sgrs', 'Prtn', 'Vtmn_A_PrDV', 'Vtmn_C_PrDV', 'Clcm_PrDV', 'Irn_PrDV', 'Srvng_mag']


Ctgry              object
Itm                object
Srvng_Sz           object
Clrs                int64
Clrs_frm_Ft         int64
Ttl_Ft            float64
Ttl_Ft_PrDV         int64
Strtd_Ft          float64
Strtd_Ft_PrDV       int64
Trns_Ft           float64
Chlstrl             int64
Chlstrl_PrDV        int64
Sdm                 int64
Sdm_PrDV            int64
Crbhydrts           int64
Crbhydrts_PrDV      int64
Dtry_Fbr            int64
Dtry_Fbr_PrDV       int64
Sgrs                int64
Prtn                int64
Vtmn_A_PrDV         int64
Vtmn_C_PrDV         int64
Clcm_PrDV           int64
Irn_PrDV            int64
Srvng_mag          object
Srvng_unit         object
dtype: object